In [19]:
import glob
import vaex
import dask
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

from typing import List
from datetime import datetime
from numpy.linalg import inv, matrix_power 
from scipy.linalg import block_diag

In [2]:
PROJECT_PATH = 'D:/work/personal/FBD_Project/'
orderbook_list = sorted(glob.glob(PROJECT_PATH + 'datasets/btcusdt/orderbooks/*.csv.gz'))
quote_list = sorted(glob.glob(PROJECT_PATH + 'datasets/btcusdt/quotes/*.csv.gz'))

In [20]:
@dask.delayed
def extract_features(orderbook_file_path:str) -> pd.DataFrame:
    df = pd.read_csv(orderbook_file_path)[['timestamp', 'asks[0].price', 'bids[0].price', 'asks[0].amount', 'bids[0].amount']]
                           
   # calculate mid price and bidask spread
    df['mid_price'] = (df['asks[0].price'] + df['bids[0].price'])/2
    df['ba_spread'] = np.round((df['asks[0].price'] - df['bids[0].price']),2)
    df['imbalance'] = df['bids[0].amount']/(df['bids[0].amount'] + df['asks[0].amount'])
    df['timestamp'] = pd.to_datetime(df['timestamp']/1000, unit='ms')

    # convert timestamp to datetime format
    df = df[['timestamp','mid_price', 'ba_spread', 'imbalance']].set_index('timestamp')
    
    # resample by 1second frequency
    df = df.resample('1s').last().ffill()
    return df


def symmetrize_data(
        df_feature: pd.DataFrame, 
        numSpreads:int=4, 
        numImbalance:int=4
    ) -> pd.DataFrame:
    
    df_signal = df_feature.copy(deep=True)
    tick_size = df_signal.ba_spread[df_signal.ba_spread != 0].min()
    
    # discretize bidask spread then get next time's bidask spread
    # discretize imbalance and get next imbalance
    # cap spread values that goes over 0.2 as 0.25 (group 5 means spread is over 0.2)
    df_signal = df_signal[df_signal.ba_spread <= numSpreads * tick_size]
    df_signal['ba_spread'] = np.round(df_signal['ba_spread'].div(tick_size)).astype(int)
    df_signal['imbalance'] = pd.cut(df_feature['imbalance'], bins=np.arange(numImbalance+1)/numImbalance, labels=np.arange(1,numImbalance+1)).astype(int)

    # calculate change in mid price
    # include data that bidask spread is within 0.2, same goes for
    # mid price change
    df_signal['mid_chg'] = np.round(df_signal['mid_price'].diff(),2).shift(-1,)
    df_signal = df_signal[abs(df_signal.mid_chg) <= 0.1]

    df_signal['next_ba_spread'] = df_signal['ba_spread'].shift(-1)
    df_signal['next_imbalance'] = df_signal['imbalance'].shift(-1)
    df_signal = df_signal.dropna()
    
    # make symmetric data 
    df_symmetric = df_signal.copy(deep=True)
    df_symmetric['imbalance'] = numImbalance - df_signal['imbalance'] + 1
    df_symmetric['next_imbalance'] = numImbalance - df_signal['next_imbalance'] + 1
    df_symmetric['mid_chg'] = -df_signal['mid_chg']

    df = pd.concat([df_signal, df_symmetric])
    df[['next_ba_spread', 'next_imbalance']] = df[['next_ba_spread', 'next_imbalance']].astype(int)
    return df.dropna()

def get_micro_adjustment(df_sig:pd.DataFrame) -> list[np.ndarray, np.ndarray]: 
    
    nSpread, nImbalance = len(df_sig.ba_spread.unique()), len(df_sig.imbalance.unique())
    nCombination = nSpread * nImbalance
    
    # divide datafrmae into two events (dM equals to 0 or not equal to 0)
    mid_zero, mid_non_zero = df_sig[df_sig['mid_chg'] == 0], df_sig[df_sig['mid_chg'] != 0]

    # transition matrix Q
    mid_zero = mid_zero.groupby(['ba_spread', 'imbalance', 'next_imbalance'])['mid_price'].count()
    Q_cnt = pd.DataFrame([],
            index=pd.MultiIndex.from_product([
                list(range(1,nSpread+1)),
                list(range(1,nImbalance+1)),
                list(range(1,nImbalance+1))
                ],
                names=['ba_spread','imbalance','next_imbalance']),
            columns=['cnt']
        ).fillna(0)

    Q_cnt.loc[mid_zero.index] = mid_zero.values.reshape(-1,1)
    Q_cnt = block_diag(
                Q_cnt.loc[1].values.reshape(nSpread,nImbalance), 
                Q_cnt.loc[2].values.reshape(nSpread,nImbalance), 
                Q_cnt.loc[3].values.reshape(nSpread,nImbalance), 
                Q_cnt.loc[4].values.reshape(nSpread,nImbalance)
            )

    # absorbing state matrix R
    R = mid_non_zero.groupby(['ba_spread','imbalance', 'mid_chg']).count().unstack('mid_chg')
    R = R['mid_price'].fillna(0).values

    # get transition matrix (transient, absorbing state)
    J = np.concatenate([Q_cnt, R],axis=1) 

    # calculate probability 
    J = J/J.sum(axis=1).reshape(-1,1)
    J = np.nan_to_num(J, nan=0)

    # split Q, R and define K
    Q, R = J[:,:nCombination], J[:,nCombination:]
    I, K = np.eye(nCombination), np.array([-0.1, -0.05, 0.05, 0.1]).reshape(-1,1)

    # 1st order micro-price adjustment
    g1 = inv(I - Q) @ R @ K

    # define new absorbing state
    T_cnt = mid_non_zero.pivot_table(
            index=['ba_spread','imbalance'],
            columns=['next_ba_spread', 'next_imbalance'],
            values='mid_price',
            fill_value=0,
            aggfunc='count'
        ).values

    J2 = np.concatenate([Q_cnt, T_cnt],axis=1)
    J2 = J2/J2.sum(axis=1).reshape(-1,1)

    # new Q and T
    Q, T = J2[:,:nCombination], J2[:,nCombination:]

    # calculate B matrix
    B = inv(I - Q) @ T
    return g1, B

In [4]:
%time
all_features = [extract_features(path) for path in orderbook_list[:20]] 

df_feat = dask.compute(all_features)[0]
df_feat = pd.concat(df_feat)

CPU times: total: 0 ns
Wall time: 0 ns


imbalance 

$ I_t = \Sigma_{j=1}^{n} j \mathbf{1}_{\frac{j-1}{n} < I < \frac{j}{n}} $

imbalance 값을 discretize 했다고 보면 됨. 10등분 한다면 $n=10$ 이 되는셈. 많이 쪼갤수록 좋을듯 

bid-ask spread $S_t$ 는 tick 단위로 count 함. 만약 tick 단위가 0.05 이고 bid-ask spread 가 0.1 이면 $S_t =2$

Discretized Markov process 사용. state space 아래와 같이 정의

$X_t = (I_t, S_t)$

In [5]:
df_sig = symmetrize_data(df_feat)

In [9]:
g1, B = get_micro_adjustment(df_sig)

In [23]:
g1

array([[-8.45811850e-03],
       [-2.20669170e-03],
       [ 2.20669170e-03],
       [ 8.45811850e-03],
       [-1.33187773e-02],
       [-3.46944695e-18],
       [ 3.46944695e-18],
       [ 1.33187773e-02],
       [-3.26115986e-02],
       [ 1.77222434e-02],
       [-1.77222434e-02],
       [ 3.26115986e-02],
       [ 1.87500000e-02],
       [ 2.50000000e-02],
       [-2.50000000e-02],
       [-1.87500000e-02]])

In [22]:
matrix_power(B, 1) @ g1

array([[-3.89342545e-04],
       [-1.02288402e-04],
       [ 1.02288402e-04],
       [ 3.89342545e-04],
       [-4.65529250e-04],
       [ 7.74002340e-04],
       [-7.74002340e-04],
       [ 4.65529250e-04],
       [-6.67509676e-04],
       [-2.69634926e-05],
       [ 2.69634926e-05],
       [ 6.67509676e-04],
       [-3.03791175e-04],
       [-2.66620255e-03],
       [ 2.66620255e-03],
       [ 3.03791175e-04]])

In [10]:
B @ g1

array([[-3.89342545e-04],
       [-1.02288402e-04],
       [ 1.02288402e-04],
       [ 3.89342545e-04],
       [-4.65529250e-04],
       [ 7.74002340e-04],
       [-7.74002340e-04],
       [ 4.65529250e-04],
       [-6.67509676e-04],
       [-2.69634926e-05],
       [ 2.69634926e-05],
       [ 6.67509676e-04],
       [-3.03791175e-04],
       [-2.66620255e-03],
       [ 2.66620255e-03],
       [ 3.03791175e-04]])

In [102]:
unique_x = list([(j,i) for i in range(1,5) for j in range(1,5)])
K = np.array([-0.1, -0.05, 0.05, 0.1])

# Q_xy: transition prob matrix for cases dM = 0
Q_xy = df_sig[df_sig.mid_chg == 0].groupby(['x_now', 'x_next']).count().unstack().fillna(0)
# Q_xy = Q_xy/Q_xy.sum(axis=1).values.reshape(-1,1)
Q_xy.columns = Q_xy.columns.droplevel(0)

# T_xy: transition prob matrix for cases dM != 0
T_xy = df_sig[df_sig.mid_chg != 0].groupby(['x_now', 'x_next']).count().unstack().fillna(0)
# T_xy = T_xy/T_xy.sum(axis=1).values.reshape(-1,1)
T_xy.columns = T_xy.columns.droplevel(0)

# R_xk: transient state matrix 
R_xk = df_sig.groupby(['x_now', 'mid_chg']).count().unstack().fillna(0)
# R_xk = R_xk/R_xk.sum(axis=1).values.reshape(-1,1)

# # ensure Q and T have shape mn x mn
Q = pd.DataFrame(0, index=unique_x, columns=unique_x, dtype=float)
Q.loc[Q_xy.columns,Q_xy.columns] = Q_xy

# T = Q.copy(deep=True)

# Q.loc[Q_xy.columns,Q_xy.columns] = Q_xy
# T.loc[T_xy.columns,T_xy.columns] = T_xy

# Q, T = Q.fillna(0), T.fillna(0)

In [103]:
Q

,"(1, 1)","(2, 1)","(3, 1)","(4, 1)","(1, 2)","(2, 2)","(3, 2)","(4, 2)","(1, 3)","(2, 3)","(3, 3)","(4, 3)","(1, 4)","(2, 4)","(3, 4)","(4, 4)"
"(1, 1)",419261.0,110164.0,61062.0,69771.0,54.0,25.0,24.0,65.0,36.0,10.0,12.0,27.0,6.0,2.0,1.0,3.0
"(2, 1)",111726.0,163099.0,71245.0,60062.0,43.0,13.0,11.0,33.0,15.0,11.0,7.0,19.0,2.0,0.0,0.0,1.0
"(3, 1)",60061.0,71245.0,163099.0,111726.0,33.0,11.0,13.0,43.0,19.0,7.0,11.0,15.0,1.0,0.0,0.0,2.0
"(4, 1)",69771.0,61062.0,110164.0,419261.0,65.0,24.0,25.0,54.0,27.0,12.0,10.0,36.0,3.0,1.0,2.0,6.0
"(1, 2)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 2)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(3, 2)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(4, 2)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 3)",4.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 3)",1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
pd.DataFrame(T1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,283692.0,106725.0,43229.0,16364.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26889.0,21.0,17.0,2494.0
1,113895.0,341723.0,122603.0,38579.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11877.0,33.0,35.0,5179.0
2,38579.0,122603.0,341723.0,113895.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5179.0,35.0,33.0,11877.0
3,16364.0,43229.0,106725.0,283692.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2494.0,17.0,21.0,26889.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,117.0,61.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,63.0,2.0
6,0.0,0.0,0.0,0.0,2.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,63.0,70.0,0.0
7,0.0,0.0,0.0,0.0,1.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,117.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,28.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,33.0,0.0,1.0,28.0


x_next,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(2, 1)","(2, 2)","(2, 3)","(2, 4)","(3, 1)","(3, 2)","(3, 3)","(3, 4)","(4, 1)","(4, 2)","(4, 3)","(4, 4)"
x_now,,,,,,,,,,,,,,,,
"(1, 1)",438801.0,22.0,21.0,1.0,106902.0,10.0,5.0,1.0,56481.0,16.0,8.0,0.0,58208.0,29.0,15.0,3.0
"(1, 2)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(1, 3)",3.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(2, 1)",110234.0,23.0,6.0,1.0,168874.0,6.0,5.0,0.0,71981.0,8.0,4.0,0.0,55113.0,21.0,11.0,0.0
"(2, 3)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(3, 1)",55177.0,12.0,8.0,1.0,71888.0,3.0,4.0,0.0,168817.0,7.0,6.0,0.0,110334.0,20.0,9.0,1.0
"(3, 3)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
"(4, 1)",57894.0,36.0,14.0,0.0,56082.0,8.0,4.0,1.0,106653.0,15.0,6.0,1.0,439755.0,32.0,17.0,5.0
"(4, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


x_next                                        
mid_chg     -0.10     -0.05      0.00      0.05      0.10
x_now                                                    
(1, 1)   0.046125  0.000052  0.948240  0.000042  0.005541
(1, 2)   0.004367  0.624454  0.004367  0.366812  0.000000
(1, 3)   0.612069  0.000000  0.094828  0.000000  0.293103
(1, 4)   0.000000  0.312500  0.000000  0.687500  0.000000
(2, 1)   0.015896  0.000043  0.974833  0.000055  0.009173
(2, 2)   0.000000  0.511628  0.000000  0.465116  0.023256
(2, 3)   0.369565  0.000000  0.086957  0.021739  0.521739
(2, 4)   0.000000  0.250000  0.000000  0.750000  0.000000
(3, 1)   0.009173  0.000055  0.974833  0.000043  0.015896
(3, 2)   0.023256  0.465116  0.000000  0.511628  0.000000
(3, 3)   0.521739  0.021739  0.086957  0.000000  0.369565
(3, 4)   0.000000  0.750000  0.000000  0.250000  0.000000
(4, 1)   0.005541  0.000042  0.948238  0.000052  0.046127
(4, 2)   0.000000  0.366812  0.004367  0.624454  0.004367
(4, 3)   0.293103  0.000000  0.094828  0.000000  0.612069
(4, 4)   0.000000  0.687500  0.000000  0.312500  0.000000

In [211]:
g_1   = np.linalg.inv(1-Q) @ R_xk @ K 
B_mat = np.linalg.inv(1-Q) @ T_xy

LinAlgError: Singular matrix

In [111]:
micro_adjustments = []
for i in range(1,20):
    micro_adjustments.append(np.linalg.matrix_power(B_mat, i) @  g_1)

In [114]:
g_1 + np.linalg.matrix_power(B_mat, 20) @  g_1

0   -1.868313e+82
1    1.034851e+88
2   -2.486767e+82
3   -2.466386e+87
4    2.462531e+82
5    2.466392e+87
6    1.844079e+82
7   -1.034851e+88
dtype: float64

In [113]:
np.array(micro_adjustments)

array([[ 3.16368358e+04, -1.72311355e+10,  4.14447507e+04,
         4.08741689e+09, -4.05255976e+04, -4.08745404e+09,
        -3.07177947e+04,  1.72311713e+10],
       [-4.10624156e+08,  2.13876552e+14, -5.38276676e+08,
        -5.09670748e+13,  4.84465666e+08,  5.09675451e+13,
         3.56813846e+08, -2.13876941e+14],
       [ 4.97423420e+12, -2.65317081e+18,  6.55977948e+12,
         6.32334519e+17, -6.12928023e+12, -6.32336823e+17,
        -4.54373353e+12,  2.65317247e+18],
       [-5.89935662e+16,  3.29124631e+22, -7.86628395e+16,
        -7.84412021e+21,  7.87449224e+16,  7.84411827e+21,
         5.90756015e+16, -3.29124613e+22],
       [ 7.18506699e+20, -4.08277260e+26,  9.62503462e+20,
         9.73058749e+25, -9.90131368e+20, -9.73058782e+25,
        -7.46134576e+20,  4.08277305e+26],
       [-9.01968982e+24,  5.06465570e+30, -1.20464599e+25,
        -1.20707267e+30,  1.21758675e+25,  1.20707540e+30,
         9.14910431e+24, -5.06465863e+30],
       [ 1.13862860e+29, -6.282676